<a href="https://colab.research.google.com/github/Noelleip/Module2_note/blob/main/defaultmodel_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import os
import pathlib
import cv2
import imghdr
import numpy as np
import matplotlib.pyplot as plt

BATCH_SIZE = 64
IMAGE_HEIGHT = 160
IMAGE_WIDTH = 160
MODEL_NAME="model"
UNET="unet_model"
MOBILE_NET_V2="mobilenet_v2"
LEARNING_RATE=0.01
INITIAL_EPOCHS = 100

In [5]:
data_dir ="datasets"
data_dir = pathlib.Path(data_dir)
image_count = len(list(data_dir.glob('*/*.jpg')))

In [3]:
class_names=list(data_dir.glob('*/'))
print(class_names)

[PosixPath('datasets/.ipynb_checkpoints'), PosixPath('datasets/2_sticky'), PosixPath('datasets/1_egg_tart')]


In [7]:
def prepare():
    global train_ds
    global val_ds

    # Loading dataset from folders
    train_ds = tf.keras.utils.image_dataset_from_directory(
        data_dir,
        validation_split=0.2,
        subset="training",
        seed=123,
        image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
        batch_size=BATCH_SIZE)
    val_ds = tf.keras.utils.image_dataset_from_directory(
        data_dir,
        validation_split=0.2,
        subset="validation",
        seed=123,
        image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
        batch_size=BATCH_SIZE)

    # Configure the dataset for performance
    AUTOTUNE = tf.data.AUTOTUNE
    train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
    val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
    return

In [10]:
def train():
    global model
    global train_ds
    global val_ds

    prepare()

    # Data augmentation
    data_augmentation = tf.keras.Sequential(
    [
        tf.keras.layers.RandomFlip("horizontal_and_vertical",
                        input_shape=(IMAGE_HEIGHT,
                                    IMAGE_WIDTH,
                                    3)),
        tf.keras.layers.RandomRotation(0.2),
        tf.keras.layers.RandomZoom(0.1),
        tf.keras.layers.RandomTranslation(0.05,0.05),
        tf.keras.layers.RandomBrightness(0.2)
    ]
    )

    # Preparing model
    num_classes = len(class_names)
    mirrored_strategy = tf.distribute.MirroredStrategy()
    with mirrored_strategy.scope():
        model = tf.keras.Sequential([
        data_augmentation,# Data augmentation Layer
            tf.keras.layers.Rescaling(1./255),# Standardize the data (0~1)
            tf.keras.layers.Conv2D(32, 3, activation='relu'),
            tf.keras.layers.MaxPooling2D(),
            tf.keras.layers.Conv2D(32, 3, activation='relu'),
            tf.keras.layers.MaxPooling2D(),
            tf.keras.layers.Conv2D(32, 3, activation='relu'),
            tf.keras.layers.MaxPooling2D(),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(180, activation='relu'),
            tf.keras.layers.Dense(num_classes)
        ])
        base_learning_rate = 0.0001
        model.compile(
            # optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
            optimizer=tf.keras.optimizers.Adam(),
            loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=['accuracy'])
        model.summary()
        history=model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=100
        )
    return

In [11]:
if __name__ == '__main__':
    train()

Found 16 files belonging to 3 classes.
Using 13 files for training.
Found 16 files belonging to 3 classes.


Using 3 files for validation.
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_2 (Sequential)   (None, 160, 160, 3)       0         
                                                                 
 rescaling_1 (Rescaling)     (None, 160, 160, 3)       0         
                                                                 
 conv2d_3 (Conv2D)           (None, 158, 158, 32)      896       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 79, 79, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 77, 77, 32)        9248      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 38, 38, 32)       0         
 2D)                    

1/1 [==============================] - 6s 6s/step - loss: 1.0800 - accuracy: 0.3846 - val_loss: 0.7324 - val_accuracy: 0.3333
Epoch 2/100
1/1 [==============================] - 0s 381ms/step - loss: 0.7166 - accuracy: 0.5385 - val_loss: 0.6504 - val_accuracy: 1.0000
Epoch 3/100
1/1 [==============================] - 0s 398ms/step - loss: 0.6761 - accuracy: 0.6923 - val_loss: 0.6112 - val_accuracy: 1.0000
Epoch 4/100
1/1 [==============================] - 0s 392ms/step - loss: 0.6557 - accuracy: 0.6154 - val_loss: 1.0374 - val_accuracy: 0.3333
Epoch 5/100
1/1 [==============================] - 0s 404ms/step - loss: 0.9422 - accuracy: 0.5385 - val_loss: 0.5731 - val_accuracy: 0.6667
Epoch 6/100
1/1 [==============================] - 0s 393ms/step - loss: 0.7787 - accuracy: 0.4615 - val_loss: 0.5558 - val_accuracy: 0.6667
Epoch 7/100
1/1 [==============================] - 0s 389ms/step - loss: 0.6278 - accuracy: 0.5385 - val_loss: 0.6262 - val_accuracy: 0.3333
Epoch 8/100
1/1 [===========